In [ ]:
import numpy as np
from enum import Enum

# Game objects

In [ ]:
class MoveException(Exception):
    pass

class Side(Enum):
    NORTH = 0
    SOUTH = 1
    def opposite(self):
        return self.NORTH if self == self.SOUTH else self.SOUTH
    
class Board(object):
    """Represents a kalah board. Responsible for validating moves and executing them."""
    
    def __init__(self, no_of_holes, no_of_seeds):
        self.no_of_holes = no_of_holes
        self.no_of_initial_seeds = no_of_seeds
        self.buckets = np.array(([no_of_seeds] * no_of_holes + [0]) * 2)
        
    def move(self, index, side):
        
        # Normal move
        self.__validate_index(index)
        no_of_seeds = self.buckets[index]
        self.buckets[index] = 0
        n = self.buckets.size
        hanging_seeds = np.concatenate((
            np.zeros(index + 1),
            np.ones(no_of_seeds),
            np.zeros((n - index - no_of_seeds - 1) % n)))
        hanging_seeds = np.sum(hanging_seeds.reshape(hanging_seeds.size // n, n), axis=0)
        self.buckets = (self.buckets + hanging_seeds).astype(int)
        
        # Grant another move if finished in store
        relative_finish_index = (index % int(n // 2) + no_of_seeds) % n
        finished_in_store = relative_finish_index == n // 2 - 1
        if finished_in_store:
            return side
        
        # Check if opposite side is zero
        finish_index = (index + no_of_seeds) % n
        opposite_index = n-finish_index-2
        finished_on_own_side =  finish_index < n // 2 if side == Side.SOUTH else finish_index > n //2
        finished_on_zero = self.buckets[finish_index] == 0
        opposite_side_has_seeds = self.buckets[opposite_index] != 0
        if finished_on_own_side and finished_on_zero and opposite_side_has_seeds:
            store_index = int(n//2)-1 if side == Side.SOUTH else n-1
            seeds_won = self.buckets[opposite_index] + 1
            self.buckets[opposite_index] = 0
            self.buckets[finish_index] = 0
            self.buckets[store_index] = self.buckets[store_index] + seeds_won   
        
        return side.opposite()
            
    def get_holes(self, side):
        start = 0 if side == Side.SOUTH else self.no_of_holes + 1
        return self.buckets[start:start+self.no_of_holes]
    
    def to_str(self):
        """Calculate matrix resembling a board's visual arrangement."""
        n = self.buckets.size
        half = int(n // 2)
        south = self.buckets[:half].reshape(1, half)
        north = np.flip(self.buckets[half:], axis=0).reshape(1, half)
        return np.concatenate((north, south), axis=0)
    
    def __validate_index(self, index):
        """Check if move at index is a valid move."""
        n = self.buckets.size
        is_store = index == int(n // 2) - 1 or index == n - 1
        if is_store:
            raise ValueError('The following index is a store: ' + str(index))  
        if self.buckets[index] == 0:
            raise ValueError('The indexed bucket is empty: ' + str(index))        

            

class State(object):
    """Acts as interface between board actions and players. Stores board and player states."""
    
    def __init__(self, no_of_holes=6, no_of_seeds=4, pie_rule=False):       
        self.players = [Side.SOUTH, Side.NORTH] # Player 1 is in position 0
        self.current_player = Side.SOUTH
        self.board = Board(no_of_holes, no_of_seeds)
        self.move_num = 1 # n-1 moves were made, nth coming up
        self.is_pie_rule_on = pie_rule
        
    def is_game_over(self):
        board_size = int(self.board.buckets.size // 2)
        south_validation = np.all(self.board.get_holes(Side.SOUTH) == 0)
        north_validation = np.all(self.board.get_holes(Side.NORTH) == 0)
        return south_validation or north_validation
    
    def move(self, index):
        if self.is_game_over():
            raise MoveException("Can't move because game is over.")
        index = self.__validate_index(index, self.current_player)
        self.current_player = self.board.move(index, self.current_player) 
        self.move_num += 1
        
    def use_pie_rule(self):
        if not self.is_pie_rule_on:
            raise MoveException("Can't use pie rule, this game is played without it.")
        if self.move_num == 2 and self.players == [Side.SOUTH, Side.NORTH]:
            self.__players_change_sides()
        else:
            raise MoveException("The game is not right after the first move, so pie rule is not applicable.")
        
    def show(self):
        print(self.board.to_str())
        
    def __change_current_player(self):
        self.current_player = self.current_player.opposite()
        
    def __players_change_sides(self):
        self.players.reverse()
        
    def __validate_index(self, index, side):
        n = self.board.buckets.size
        half = int(n // 2)
        out_of_bounds = index < 0 or index >= half
        if out_of_bounds:
            raise ValueError('The following index is out of bounds: ' + str(index))
        return index if side == Side.SOUTH else index + half
        
        
        

# Example game

In [ ]:
random_game = False
no_of_holes = 6
S = State(no_of_holes,4,pie_rule=True)

if random_game:
    explorable_moves = list(range(no_of_holes+1))
    while True:
        move_index = np.random.choice(explorable_moves)
        try:
            S.move(move_index)
        except ValueError:
            explorable_moves.remove(move_index)
        except MoveException:
            print("Game ended. Final position:")
            S.show()
            break
        else:
            explorable_moves = list(range(no_of_holes+1))
else:    
    predefined_moves = [2, 5, 3, 2, 5, 5, 1, 1, 0]
    for i,move in enumerate(moves):
        if i==1:
            S.use_pie_rule()
        S.move(move)
        S.show()
        print()

In [ ]:
print(S.players)